# Write the necessary weather data from the given .csv file

In [ ]:
# imports -- change ip and fingerprint between hosts
import pandas as pd
import pyexasol

host = ''   # Don't forget to change the fingerprint
user = 'sys'
password = 'exasol'

# Connect to Exasol
conn = pyexasol.connect(dsn=host, 
                        user=user, 
                        password=password, 
                        debug=False, 
                        protocol_version=pyexasol.PROTOCOL_V1)

#### Read in CSV

In [2]:
df = pd.read_csv('./../Data/Stormdata_2006.csv', encoding='iso-8859-1')
print(df.shape)
print(df.columns)

# Convert start and end dates to date time
df['END_DATE_TIME'] = pd.to_datetime(df['END_DATE_TIME'])
df['BEGIN_DATE_TIME'] = pd.to_datetime(df['BEGIN_DATE_TIME'])

print(df.shape)
print("min:", min(df['BEGIN_DATE_TIME']), "\nmax:", max(df['END_DATE_TIME']))

# Remove those ending after our AOL database - Those before could be ok?
df = df[df['END_DATE_TIME'] <= '2006-06-01 00:00:00']

print(df.shape)
print("min:", min(df['BEGIN_DATE_TIME']), "\nmax:", max(df['END_DATE_TIME']))

(48595, 58)
Index(['BEGIN_YEARMONTH', 'BEGIN_DAY', 'BEGIN_TIME', 'END_YEARMONTH',
       'END_DAY', 'END_TIME', 'EPISODE_ID', 'EVENT_ID', 'STATE', 'STATE_FIPS',
       'YEAR', 'MONTH_NAME', 'EVENT_TYPE', 'CZ_TYPE', 'CZ_FIPS', 'CZ_NAME',
       'WFO', 'BEGIN_DATE_TIME', 'CZ_TIMEZONE', 'END_DATE_TIME',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'SOURCE',
       'MAGNITUDE', 'MAGNITUDE_TYPE', 'FLOOD_CAUSE', 'CATEGORY', 'TOR_F_SCALE',
       'TOR_LENGTH', 'TOR_WIDTH', 'TOR_OTHER_WFO', 'TOR_OTHER_CZ_STATE',
       'TOR_OTHER_CZ_FIPS', 'TOR_OTHER_CZ_NAME', 'BEGIN_RANGE',
       'BEGIN_AZIMUTH', 'BEGIN_LOCATION', 'END_RANGE', 'END_AZIMUTH',
       'END_LOCATION', 'BEGIN_LAT', 'BEGIN_LON', 'END_LAT', 'END_LON',
       'EPISODE_NARRATIVE', 'EVENT_NARRATIVE', 'LAST_MOD_DATE',
       'LAST_MOD_TIME', 'LAST_CERT_DATE', 'LAST_CERT_TIME', 'LAST_MOD',
       'LAST_CERT', 'ADDCORR_FLG', 'ADDCORR_DATE'],
      dtype='obje

#### Drop irrelevant columns

In [3]:
# Keep: 'BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE', 'BEGIN_DAY', 'END_DAY', 'BEGIN_YEARMONTH', 'END_YEARMONTH', 'STATE', 'STATE_FIPS', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS' 
keep = ['BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE', 'BEGIN_DAY', 'END_DAY', 'BEGIN_YEARMONTH', 'END_YEARMONTH', 'STATE', 'STATE_FIPS', 'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'EPISODE_ID', 'EVENT_ID']

df = df.loc[:, keep]

print(df.columns)
print(df.head)

# Fix year/month and naming oddness
df['BEGIN_MONTH'] = df['BEGIN_YEARMONTH'] - 200600
df['END_MONTH'] = df['END_YEARMONTH'] - 200600
df['REGION'] = df['STATE'].str[0].str.upper() + df['STATE'].str[1:].str.lower() # This contains all the spatial information we need- writing the state name was buggy
df['EVENT_TYPE'] = df['EVENT_TYPE'].astype(str)

df = df.drop(columns= ['BEGIN_YEARMONTH', 'END_YEARMONTH', 'STATE'])

print(df.columns)
print(df.head)


Index(['BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE', 'BEGIN_DAY',
       'END_DAY', 'BEGIN_YEARMONTH', 'END_YEARMONTH', 'STATE', 'STATE_FIPS',
       'INJURIES_DIRECT', 'INJURIES_INDIRECT', 'DEATHS_DIRECT',
       'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS', 'EPISODE_ID',
       'EVENT_ID'],
      dtype='object')
<bound method NDFrame.head of           BEGIN_DATE_TIME       END_DATE_TIME                EVENT_TYPE  \
0     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
1     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
2     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
3     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
4     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
...                   ...                 ...                       ...   
25729 2006-05-31 21:10:00 2006-05-31 22:15:00  Marine Thunderstorm Wind   
25730 2006-05-31 22:00:00 2006-05-31 22:00

#### Clean up the damage cols

In [4]:
def convert_abbreviated_string(value):
    # Check for 'k' (thousand), 'm' (million), 'b' (billion)
    if isinstance(value, str):
        print(value)
        if 'k' in value.lower():
            return float(value.replace('k', '').replace('K', '')) * 1000
        elif 'm' in value.lower():
            return float(value.replace('m', '').replace('M', '')) * 1000000
        elif 'b' in value.lower():
            return float(value.replace('b', '').replace('B', '')) * 1000000000
        else:
            # If no abbreviation, just return the float version of the number
            try:
                return float(value)
            except ValueError:
                return None  # or handle invalid strings as needed
    return value

In [5]:

# Print the unique columns and the one anomaly
print(df[df['DAMAGE_PROPERTY'].notna()]['DAMAGE_PROPERTY'].unique())
print(sum(df[df['DAMAGE_PROPERTY'].notna()]['DAMAGE_PROPERTY']=='K'))

print(df[df['DAMAGE_CROPS'].notna()]['DAMAGE_CROPS'].unique())

['900K' '4.9M' '2M' '3.2M' '4.5M' '700K' '100K' '1M' '15M' '165K' '243K'
 '524K' '623K' '1.2M' '5M' '115B' '104M' '0' '22M' '2.5M' '8.8M' '108M'
 '70K' '20K' '50K' '35K' '36K' '87K' '15K' '175K' '0K' '500K' '200K'
 '800K' '1K' '10K' '2K' '25K' '71K' '1.4M' '5.5M' '.1K' '75K' '5K' '45K'
 '55K' '.5K' '3K' '750K' '4M' '250K' '150K' '350K' '8K' '300K' '400K' '7K'
 '12K' '80K' '2.9M' '600K' '130K' '30K' '90K' '450K' '4K' '1.5K' '65K'
 '60K' '40K' '.25M' '120K' '.1M' '.5M' '160K' '16K' '2.5K' '.05K' '81K'
 '9K' '7.5M' '230K' '125K' '94.5K' '380K' '14K' '1.6M' '6K' '4.5K' '.01K'
 '1.5M' '85K' '11K' '168K' '357K' '.2K' '.25K' '10M' '.21K' '.17K' '8M'
 '19.9M' '30M' '3M' '850K' '50M' '650K' '3.5K' '126K' '265K' '.86K' '27K'
 '34K' '.75K' '190K' '240K' '43K' '210K' '21K' '.3K' '590K' '164K' '1.1M'
 '.04M' '.01M' '.15K' '.85K' '294K' '632K' '245K' '146K' '1.37M' '1.59M'
 '932K' '237K' '975K' '887K' '113K' '1.34M' '1.14M' '1.32M' '109K' '177K'
 '996K' '552K' '100M' '7M' '14.4M' '11.5M' '18K' '25M'

#### Modify the original DF

In [6]:
# Clean Up the damage property/crops data
df['DAMAGE_PROPERTY'] = df[df['DAMAGE_PROPERTY'] == 'K']['DAMAGE_PROPERTY'] = 1000  # Reporting error

# Get the string abbr. out
df['DAMAGE_CROPS'] = df['DAMAGE_CROPS'].replace("", float('nan')).apply(convert_abbreviated_string)
df['DAMAGE_PROPERTY'] = df['DAMAGE_PROPERTY'].replace("", float('nan')).apply(convert_abbreviated_string)


1B
32.5M
3M
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
750K
0
0
0
0
0
0
0
0
10K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1.3M
0
0K
0
0
0
0
0
0
0
0
1M
0
0
0
0
0
0
0
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1M
0
0
0
0
0
0
0
0
0
10K
10K
0
0
30K
0
0
0
0
0
0
0
0
0
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
50K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0
0
0
0
0
3K
0K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
15K
0
0
0
0
0
0
500K
300M
0
10K
2K
5K
0
0
0
30K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0K
0K
0K
0K
0K
0K
0K
0
0
0
0
0K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
3.9M
1.52M
4.02M
2.5M
0K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0K
0
0K
0K
0K
0K
0K
1K
112.5K
0
0
0
0
0
0
200M
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
500K
0
0
0
250K
0
0
0
0
5K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
0K
100K
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


#### Create The DB Schema on exasol

In [7]:
create_table_query = '''
CREATE TABLE AOL_SCHEMA.WEATHER_EVENTS (
    BEGIN_DATE_TIME TIMESTAMP,
    END_DATE_TIME TIMESTAMP,
    EVENT_TYPE VARCHAR(100),
    BEGIN_DAY INTEGER,
    END_DAY INTEGER,
    STATE_FIPS INTEGER,
    INJURIES_DIRECT INTEGER,
    INJURIES_INDIRECT INTEGER,
    DEATHS_DIRECT INTEGER,
    DEATHS_INDIRECT INTEGER,
    DAMAGE_PROPERTY INTEGER,
    DAMAGE_CROPS FLOAT,
    EPISODE_ID INT,
    EVENT_ID INT NOT NULL,
    BEGIN_MONTH INTEGER,
    END_MONTH INTEGER,
    REGION VARCHAR(100),
    PRIMARY KEY (EVENT_ID)
)
'''
# conn.execute("DROP TABLE AOL_SCHEMA.WEATHER_EVENTS")
conn.execute(create_table_query)

ExaQueryError: 
(
    message     =>  object WEATHER_EVENTS already exists (Session: 1816733996904888337)
    dsn         =>  192.168.56.101/E2B01857E4275EFD71F318801EA24D42A90C9EE2EE8F6A9899662520465CF79D:8563
    user        =>  sys
    schema      =>  
    session_id  =>  1816733996904888337
    code        =>  42500
    query       =>  CREATE TABLE AOL_SCHEMA.WEATHER_EVENTS (
    BEGIN_DATE_TIME TIMESTAMP,
    END_DATE_TIME TIMESTAMP,
    EVENT_TYPE VARCHAR(100),
    BEGIN_DAY INTEGER,
    END_DAY INTEGER,
    STATE_FIPS INTEGER,
    INJURIES_DIRECT INTEGER,
    INJURIES_INDIRECT INTEGER,
    DEATHS_DIRECT INTEGER,
    DEATHS_INDIRECT INTEGER,
    DAMAGE_PROPERTY INTEGER,
    DAMAGE_CROPS FLOAT,
    EPISODE_ID INT,
    EVENT_ID INT NOT NULL,
    BEGIN_MONTH INTEGER,
    END_MONTH INTEGER,
    REGION VARCHAR(100),
    PRIMARY KEY (EVENT_ID)
)
)


In [8]:
query = "DESCRIBE AOL_SCHEMA.WEATHER_EVENTS"  # Replace with your schema and table name
result = conn.execute(query)

# Fetch and print the results
for row in result:
    print(row)

('BEGIN_DATE_TIME', 'TIMESTAMP', 'TRUE', 'FALSE')
('END_DATE_TIME', 'TIMESTAMP', 'TRUE', 'FALSE')
('EVENT_TYPE', 'VARCHAR(100) UTF8', 'TRUE', 'FALSE')
('BEGIN_DAY', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('END_DAY', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('STATE_FIPS', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('INJURIES_DIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('INJURIES_INDIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DEATHS_DIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DEATHS_INDIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DAMAGE_PROPERTY', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DAMAGE_CROPS', 'DOUBLE', 'TRUE', 'FALSE')
('EPISODE_ID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('EVENT_ID', 'DECIMAL(18,0)', 'FALSE', 'FALSE')
('BEGIN_MONTH', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('END_MONTH', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('REGION', 'VARCHAR(100) UTF8', 'TRUE', 'FALSE')


#### Write the data to the database

In [9]:
df.columns

Index(['BEGIN_DATE_TIME', 'END_DATE_TIME', 'EVENT_TYPE', 'BEGIN_DAY',
       'END_DAY', 'STATE_FIPS', 'INJURIES_DIRECT', 'INJURIES_INDIRECT',
       'DEATHS_DIRECT', 'DEATHS_INDIRECT', 'DAMAGE_PROPERTY', 'DAMAGE_CROPS',
       'EPISODE_ID', 'EVENT_ID', 'BEGIN_MONTH', 'END_MONTH', 'REGION'],
      dtype='object')

In [16]:
# Identify duplicates based on a specific column (e.g., 'EVENT_ID')
duplicates_df = df[df.duplicated(subset='EPISODE_ID', keep=False)]

# Display the duplicates
print(duplicates_df)


          BEGIN_DATE_TIME       END_DATE_TIME                EVENT_TYPE  \
0     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
1     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
2     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
3     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
4     2006-01-01 00:00:00 2006-01-31 23:59:00                   Drought   
...                   ...                 ...                       ...   
25728 2006-05-31 21:10:00 2006-05-31 22:10:00  Marine Thunderstorm Wind   
25729 2006-05-31 21:10:00 2006-05-31 22:15:00  Marine Thunderstorm Wind   
25730 2006-05-31 22:00:00 2006-05-31 22:00:00         Thunderstorm Wind   
25731 2006-05-31 22:05:00 2006-05-31 22:05:00         Thunderstorm Wind   
25732 2006-05-31 23:05:00 2006-05-31 23:05:00         Thunderstorm Wind   

       BEGIN_DAY  END_DAY  STATE_FIPS  INJURIES_DIRECT  INJURIES_INDIRECT  \
0              1      

In [17]:
df[df['EPISODE_ID']==202408]

,BEGIN_DATE_TIME,END_DATE_TIME,EVENT_TYPE,BEGIN_DAY,END_DAY,STATE_FIPS,INJURIES_DIRECT,INJURIES_INDIRECT,DEATHS_DIRECT,DEATHS_INDIRECT,DAMAGE_PROPERTY,DAMAGE_CROPS,EPISODE_ID,EVENT_ID,BEGIN_MONTH,END_MONTH,REGION
0,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482479,1,1,Colorado
201,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482474,1,1,Colorado
202,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482473,1,1,Colorado
203,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482472,1,1,Colorado
204,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482471,1,1,Colorado
291,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482470,1,1,Colorado
292,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482469,1,1,Colorado
293,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482467,1,1,Colorado
295,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482468,1,1,Colorado
312,2006-01-01,2006-01-31 23:59:00,Drought,1,31,8,0,0,0,0,1000,NaN,202408,5482478,1,1,Colorado


In [18]:

# Replace NaN values with None
df = df.where(pd.notnull(df), None)
print(df.shape)

# Remove duplicates based on all columns except 'EVENT_ID'
columns_except_event_id = df.columns[df.columns != 'EVENT_ID']

# Drop duplicates based on all columns except 'EVENT_ID'
distinct_df = df.drop_duplicates(subset=columns_except_event_id)

# Check the result
print(distinct_df.shape)


(25734, 17)
(19882, 17)


In [19]:
conn.import_from_pandas(distinct_df, table=('AOL_SCHEMA', 'WEATHER_EVENTS'))

In [22]:
query = "DESCRIBE AOL_SCHEMA.WEATHER_EVENTS"  # Replace with your schema and table name
result = conn.execute(query)

# Fetch and print the results
for row in result:
    print(row)

('BEGIN_DATE_TIME', 'TIMESTAMP', 'TRUE', 'FALSE')
('END_DATE_TIME', 'TIMESTAMP', 'TRUE', 'FALSE')
('EVENT_TYPE', 'VARCHAR(100) UTF8', 'TRUE', 'FALSE')
('BEGIN_DAY', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('END_DAY', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('STATE_FIPS', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('INJURIES_DIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('INJURIES_INDIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DEATHS_DIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DEATHS_INDIRECT', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DAMAGE_PROPERTY', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('DAMAGE_CROPS', 'DOUBLE', 'TRUE', 'FALSE')
('EPISODE_ID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('EVENT_ID', 'DECIMAL(18,0)', 'FALSE', 'FALSE')
('BEGIN_MONTH', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('END_MONTH', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('REGION', 'VARCHAR(100) UTF8', 'TRUE', 'FALSE')


In [23]:
# Define the SQL query
query = """
SELECT 
    BEGIN_DATE_TIME, 
    EVENT_TYPE, 
    REGION, 
    INJURIES_DIRECT, 
    INJURIES_INDIRECT, 
    DEATHS_DIRECT, 
    DEATHS_INDIRECT, 
    DAMAGE_PROPERTY, 
    DAMAGE_CROPS
FROM AOL_SCHEMA.WEATHER_EVENTS
LIMIT 10;
"""

result = conn.execute(query)

# Fetch and print the results
for row in result:
    print(row)

('2006-01-01 00:00:00.000000', 'Drought', 'Colorado', 0, 0, 0, 0, 1000, None)
('2006-01-01 00:00:00.000000', 'Drought', 'Texas', 0, 0, 0, 0, 1000, None)
('2006-01-01 00:00:00.000000', 'Drought', 'Oklahoma', 0, 0, 0, 0, 1000, None)
('2006-01-01 00:00:00.000000', 'WINTER WEATHER', 'Colorado', 0, 0, 0, 0, 1000, None)
('2006-01-01 00:00:00.000000', 'WINTER WEATHER', 'Utah', 0, 0, 0, 0, 1000, None)
('2006-01-01 00:00:00.000000', 'Drought', 'Texas', 0, 0, 0, 0, 1000, None)
('2006-01-01 00:00:00.000000', 'Drought', 'Texas', 0, 0, 0, 0, 1000, 1000000000)
('2006-01-01 00:00:00.000000', 'Drought', 'Oklahoma', 0, 0, 0, 0, 1000, None)
('2006-01-01 00:00:00.000000', 'Drought', 'Arkansas', 0, 0, 0, 0, 1000, None)
('2006-01-01 00:00:00.000000', 'Heat', 'New york', 0, 0, 0, 0, 1000, None)


In [29]:
# Query to describe the FACTS table
describe_query = "DESCRIBE AOL_SCHEMA.FACTS"

# Execute and print the results
result = conn.execute(describe_query)
for row in result:
    print(row)


('QUERYID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('TIMEID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('ANONID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('URLID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('IRANK', 'VARCHAR(100) UTF8', 'TRUE', 'FALSE')
('CLICK', 'BOOLEAN', 'TRUE', 'FALSE')


In [31]:
# List of tables to describe
tables = ["AOL_SCHEMA.FACTS", "AOL_SCHEMA.URLDIM", "AOL_SCHEMA.DMOZ_CATEGORIES", "AOL_SCHEMA.WEATHER_EVENTS"]

# Describe each table and print the results
for table in tables:
    print(f"Describing table: {table}")
    describe_query = f"DESCRIBE {table}"
    result = conn.execute(describe_query)
    for row in result:
        print(row)
    print("\n")  # Add space between table descriptions


Describing table: AOL_SCHEMA.FACTS
('QUERYID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('TIMEID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('ANONID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('URLID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('IRANK', 'VARCHAR(100) UTF8', 'TRUE', 'FALSE')
('CLICK', 'BOOLEAN', 'TRUE', 'FALSE')


Describing table: AOL_SCHEMA.URLDIM
('URL', 'VARCHAR(5000) UTF8', 'TRUE', 'FALSE')
('ID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')
('TITLE', 'VARCHAR(1000) UTF8', 'TRUE', 'FALSE')
('DESCRIPTION', 'VARCHAR(2000000) UTF8', 'TRUE', 'FALSE')
('PROTOCOL', 'VARCHAR(50) UTF8', 'TRUE', 'FALSE')
('SUBDOMAIN', 'VARCHAR(1000) UTF8', 'TRUE', 'FALSE')
('THISDOMAIN', 'VARCHAR(2500) UTF8', 'TRUE', 'FALSE')
('TOPLEVELDOMAIN', 'VARCHAR(100) UTF8', 'TRUE', 'FALSE')
('THISPATH', 'VARCHAR(2000) UTF8', 'TRUE', 'FALSE')


Describing table: AOL_SCHEMA.DMOZ_CATEGORIES
('CATID', 'DECIMAL(36,0)', 'TRUE', 'FALSE')
('TOPIC', 'VARCHAR(512) UTF8', 'TRUE', 'FALSE')
('TITLE', 'VARCHAR(256) UTF8', 'TRUE', 'FALSE')
('DESCRIPTION

In [34]:
tables = ["AOL_SCHEMA.FACTS", "AOL_SCHEMA.URLDIM", "AOL_SCHEMA.DMOZ_CATEGORIES", "AOL_SCHEMA.WEATHER_EVENTS"]

for table in tables:
    query = f"SELECT COUNT(*) AS row_count FROM {table}"
    result = conn.execute(query)
    print(f"{table}: {result.fetchone()}")


AOL_SCHEMA.FACTS: (36389566,)
AOL_SCHEMA.URLDIM: (4051750,)
AOL_SCHEMA.DMOZ_CATEGORIES: (779122,)
AOL_SCHEMA.WEATHER_EVENTS: (19882,)


In [35]:
query = """
SELECT f.URLID, u.ID, u.THISDOMAIN
FROM AOL_SCHEMA.FACTS f
JOIN AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
LIMIT 10;
"""
result = conn.export_to_pandas(query)
print(result)


   URLID    ID THISDOMAIN
0   3649  3649   thetrain
1   3649  3649   thetrain
2   3649  3649   thetrain
3   3649  3649   thetrain
4   3649  3649   thetrain
5   3649  3649   thetrain
6   3649  3649   thetrain
7   3649  3649   thetrain
8   3649  3649   thetrain
9   3649  3649   thetrain


In [37]:
query = """
SELECT DISTINCT THISDOMAIN
FROM AOL_SCHEMA.URLDIM
WHERE THISDOMAIN IS NOT NULL
LIMIT 10;
"""
result = conn.export_to_pandas(query)
print("Distinct THISDOMAIN values in URLDIM:")
print(result)


Distinct THISDOMAIN values in URLDIM:
            THISDOMAIN
0        vibratormotor
1     projectaware.org
2         buchetcetera
3             tdivadlo
4        onlineclasses
5           maresdream
6     beautyonline.com
7       mnnewspapernet
8  thingsnigerianslove
9              xpclean


In [38]:
query = """
SELECT DISTINCT TOPIC
FROM AOL_SCHEMA.DMOZ_CATEGORIES
WHERE TOPIC IS NOT NULL
LIMIT 10;
"""
result = conn.export_to_pandas(query)
print("Distinct TOPIC values in DMOZ_CATEGORIES:")
print(result)


Distinct TOPIC values in DMOZ_CATEGORIES:
                                               TOPIC
0  Top/Sports/Skating/Ice_Skating/Events/National...
1  Top/World/Greek/Κατά_Περιοχή/Ευρώπη/Ελλάδα/Νομ...
2                  Top/Home/Family/Family_Websites/N
3  Top/World/Russian/Страны_и_регионы/Европа/Росс...
4  Top/Regional/Europe/United_Kingdom/England/Ess...
5  Top/Recreation/Autos/Makes_and_Models/Nissan/2...
6     Top/World/Svenska/Kultur/Scenkonst/Dans/Balett
7        Top/World/Català/Arts/Música/Estils/Country
8  Top/Regional/North_America/United_States/Utah/...
9  Top/Recreation/Outdoors/Organizations/North_Am...


In [39]:
query = """
SELECT u.THISDOMAIN, c.TOPIC
FROM AOL_SCHEMA.URLDIM u
JOIN AOL_SCHEMA.DMOZ_CATEGORIES c ON LOWER(u.THISDOMAIN) = LOWER(TRIM(c.TOPIC))
LIMIT 10;
"""
result = conn.export_to_pandas(query)
print(result)


Empty DataFrame
Columns: [THISDOMAIN, TOPIC]
Index: []


In [ ]:
query = """
SELECT 
    u.THISDOMAIN AS DOMAIN_NAME,
    c.TOPIC AS FULL_TOPIC_PATH,
    REGEXP_SUBSTR(c.TOPIC, '[^/]+$', 1, 1) AS EXTRACTED_TOPIC, -- Extract the last segment of the topic
    CASE 
        WHEN LOWER(u.THISDOMAIN) = LOWER(REGEXP_SUBSTR(c.TOPIC, '[^/]+$', 1, 1)) THEN 'Exact Match'
        WHEN LOWER(c.TOPIC) LIKE CONCAT('%', LOWER(u.THISDOMAIN), '%') THEN 'Partial Match'
        ELSE 'No Match'
    END AS MATCH_TYPE
FROM AOL_SCHEMA.URLDIM u
LEFT JOIN AOL_SCHEMA.DMOZ_CATEGORIES c 
ON LOWER(u.THISDOMAIN) = LOWER(REGEXP_SUBSTR(c.TOPIC, '[^/]+$', 1, 1))
   OR LOWER(c.TOPIC) LIKE CONCAT('%', LOWER(u.THISDOMAIN), '%')
LIMIT 20;
"""
result = conn.export_to_pandas(query)
print(result)


In [41]:
query_weather = """
SELECT 
    EVENT_TYPE,
    REGION,
    SUM(DAMAGE_PROPERTY) AS total_property_damage,
    SUM(DEATHS_DIRECT + DEATHS_INDIRECT) AS total_fatalities
FROM AOL_SCHEMA.WEATHER_EVENTS
GROUP BY EVENT_TYPE, REGION
ORDER BY total_property_damage DESC, total_fatalities DESC;
"""
df_weather = conn.export_to_pandas(query_weather)
print("Weather Events Analysis:")
print(df_weather)


Weather Events Analysis:
          EVENT_TYPE    REGION  TOTAL_PROPERTY_DAMAGE  TOTAL_FATALITIES
0               Hail  Missouri                1051000                 0
1               Hail     Texas                 968000                 0
2               Hail    Kansas                 745000                 0
3               Hail  Illinois                 611000                 0
4               Hail  Oklahoma                 542000                 0
..               ...       ...                    ...               ...
694       Dust Storm     Texas                   1000                 0
695      Flash Flood  Nebraska                   1000                 0
696  Cold/Wind Chill     Maine                   1000                 0
697     Funnel Cloud  Oklahoma                   1000                 0
698   WINTER WEATHER  Oklahoma                   1000                 0

[699 rows x 4 columns]


In [45]:
query_facts = """
SELECT 
    u.THISDOMAIN AS domain_name,  -- Changed alias to domain_name
    u.TITLE AS website_title,
    COUNT(f.QUERYID) AS total_queries
FROM AOL_SCHEMA.FACTS f
JOIN AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
GROUP BY u.THISDOMAIN, u.TITLE
ORDER BY total_queries DESC
LIMIT 10;
"""
df_facts = conn.export_to_pandas(query_facts)
print("Website Categories Analysis based on URLDIM:")
print(df_facts)


Website Categories Analysis based on URLDIM:
  DOMAIN_NAME               WEBSITE_TITLE  TOTAL_QUERIES
0         NaN                         NaN       16947036
1      google            கூகிள் ( Google)         366623
2     myspace        MySpace: Rata Blanca         167070
3       yahoo                      Yahoo!         161082
4   wikipedia  Vladimir Lenin - Wikipedia         122539
5      amazon                      Amazon         106119
6        imdb     imdb.de: Knight and Day          98549
7    mapquest                    MapQuest          96215
8        ebay                        eBay          77962
9       yahoo                 Yahoo! Mail          53856


In [46]:
query_facts_keywords = """
SELECT 
    u.THISDOMAIN AS domain_name,  -- Use domain_name as alias
    u.TITLE AS website_title,
    COUNT(f.QUERYID) AS total_queries
FROM AOL_SCHEMA.FACTS f
JOIN AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
WHERE LOWER(u.TITLE) LIKE '%weather%' OR LOWER(u.TITLE) LIKE '%emergency%' OR LOWER(u.TITLE) LIKE '%flood%'
GROUP BY u.THISDOMAIN, u.TITLE
ORDER BY total_queries DESC
LIMIT 10;
"""
df_facts_keywords = conn.export_to_pandas(query_facts_keywords)
print("Website Analysis with Event Keywords:")
print(df_facts_keywords)


Website Analysis with Event Keywords:
          DOMAIN_NAME                    WEBSITE_TITLE  TOTAL_QUERIES
0             weather              The Weather Channel          13959
1        wunderground              Weather underground           9824
2            nws.noaa  National Weather Service (NNAA)           3372
3         accuweather                      AccuWeather           2785
4               yahoo          Yahoo! Weather - Brazil           2475
5                noaa      NOAA NWS - Colombia Weather           1187
6  weatherchannel.com              The Weather Channel           1075
7            srh.noaa         National Weather Service            934
8                noaa         NOAA Weather Information            754
9        hamptonroads            Hampton Roads weather            628


In [7]:
query_facts_keywords = """
SELECT
    w.EVENT_TYPE,
    w.REGION,
    u.THISDOMAIN AS DOMAIN_NAME,
    u.TITLE AS WEBSITE_TITLE,
    SUM(w.DAMAGE_PROPERTY) AS TOTAL_PROPERTY_DAMAGE,
    SUM(w.DEATHS_DIRECT + w.DEATHS_INDIRECT) AS TOTAL_FATALITIES,
    COUNT(f.QUERYID) AS TOTAL_QUERIES
FROM
    AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN
    AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID -- Assuming URLID maps to STATE_FIPS
LEFT JOIN
    AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
WHERE
    LOWER(u.TITLE) LIKE '%weather%'
    OR LOWER(u.TITLE) LIKE '%emergency%'
    OR LOWER(u.TITLE) LIKE '%flood%'
GROUP BY
    w.EVENT_TYPE, w.REGION, u.THISDOMAIN, u.TITLE
ORDER BY
    TOTAL_PROPERTY_DAMAGE DESC,
    TOTAL_FATALITIES DESC,
    TOTAL_QUERIES DESC
LIMIT 20;
"""
df_facts_keywords = conn.export_to_pandas(query_facts_keywords)
print("Website Analysis with Event Keywords:")
print(df_facts_keywords)


Website Analysis with Event Keywords:
Empty DataFrame
Columns: [EVENT_TYPE, REGION, DOMAIN_NAME, WEBSITE_TITLE, TOTAL_PROPERTY_DAMAGE, TOTAL_FATALITIES, TOTAL_QUERIES]
Index: []


In [23]:
query_facts_keywords = """
SELECT COUNT(*) AS DOMAIN_TOPIC_MATCH
FROM AOL_SCHEMA.URLDIM u
JOIN AOL_SCHEMA.DMOZ_CATEGORIES c 
ON u.THISDOMAIN = REGEXP_SUBSTR(c.TOPIC, '[^/]+$', 1, 1);
"""
df_facts_keywords = conn.export_to_pandas(query_facts_keywords)
print("Website Analysis with Event Keywords:")
print(df_facts_keywords)


Website Analysis with Event Keywords:
   DOMAIN_TOPIC_MATCH
0                5854


In [9]:
query_facts_keywords = """
SELECT DISTINCT STATE_FIPS
FROM AOL_SCHEMA.WEATHER_EVENTS
LIMIT 10;
"""
df_facts_keywords = conn.export_to_pandas(query_facts_keywords)
print("Website Analysis with Event Keywords:")
print(df_facts_keywords)


Website Analysis with Event Keywords:
   STATE_FIPS
0           1
1           2
2           4
3           5
4           6
5           8
6           9
7          10
8          11
9          12


In [10]:
query_facts_keywords = """
SELECT DISTINCT URLID
FROM AOL_SCHEMA.FACTS
LIMIT 10;
"""
df_facts_keywords = conn.export_to_pandas(query_facts_keywords)
print("Website Analysis with Event Keywords:")
print(df_facts_keywords)


Website Analysis with Event Keywords:
   URLID
0      1
1      2
2      3
3      4
4      5
5      6
6      7
7      8
8      9
9     10


In [ ]:
query_facts_keywords = """
SELECT
    w.EVENT_TYPE,
    w.REGION,
    u.THISDOMAIN AS DOMAIN_NAME,
    u.TITLE AS WEBSITE_TITLE,
    SUM(w.DAMAGE_PROPERTY) AS TOTAL_PROPERTY_DAMAGE,
    SUM(w.DEATHS_DIRECT + w.DEATHS_INDIRECT) AS TOTAL_FATALITIES,
    COUNT(f.QUERYID) AS TOTAL_QUERIES
FROM
    AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN
    AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
LEFT JOIN
    AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
GROUP BY w.EVENT_TYPE, w.REGION, u.THISDOMAIN, u.TITLE
ORDER BY TOTAL_PROPERTY_DAMAGE DESC
LIMIT 10;
"""
df_facts_keywords = conn.export_to_pandas(query_facts_keywords)
print("Website Analysis with Event Keywords:")
print(df_facts_keywords)


Website Analysis with Event Keywords:
          EVENT_TYPE     REGION       DOMAIN_NAME         WEBSITE_TITLE  \
0               Hail     Kansas             crime                   NaN   
1               Hail  Tennessee    buildingtrades                   NaN   
2  Thunderstorm Wind  Tennessee    buildingtrades                   NaN   
3               Hail   Illinois          mybrotha          Mybrotha.com   
4          High Wind     Oregon        watsonrent                   NaN   
5               Hail    Indiana  elliscountypress    Ellis County Press   
6               Hail  Wisconsin    kingdomhearts3      Kingdom Hearts 3   
7               Hail   Michigan       lvbeethoven  Ludwig van Beethoven   
8  Thunderstorm Wind   Illinois          mybrotha          Mybrotha.com   
9  Thunderstorm Wind     Kansas             crime                   NaN   

   TOTAL_PROPERTY_DAMAGE  TOTAL_FATALITIES  TOTAL_QUERIES  
0                8195000                 0           8195  
1               

In [25]:
query_facts_keywords = """
SELECT
    w.EVENT_TYPE,
    w.REGION,
    u.THISDOMAIN AS DOMAIN_NAME,
    u.TITLE AS WEBSITE_TITLE,
    q.QUERY AS SEARCH_QUERY,
    c.TOPIC AS CATEGORY_TOPIC
FROM
    AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN
    AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
LEFT JOIN
    AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
LEFT JOIN
    AOL_SCHEMA.QUERYDIM q ON f.QUERYID = q.ID
LEFT JOIN
    AOL_SCHEMA.DMOZ_CATEGORIES c ON u.THISDOMAIN = REGEXP_SUBSTR(c.TOPIC, '[^/]+$', 1, 1)
LIMIT 10;
"""
df_facts_keywords = conn.export_to_pandas(query_facts_keywords)
print("Website Analysis with Event Keywords:")
print(df_facts_keywords)

Website Analysis with Event Keywords:
       EVENT_TYPE          REGION       DOMAIN_NAME   WEBSITE_TITLE  \
0  Winter Weather  North carolina        kostajnica  Kostajnica.com   
1  Winter Weather  North carolina        kostajnica  Kostajnica.com   
2  Winter Weather  North carolina        kostajnica  Kostajnica.com   
3  Winter Weather  North carolina        kostajnica  Kostajnica.com   
4    Winter Storm         Alabama  darrellmansfield             NaN   
5    Winter Storm         Alabama  darrellmansfield             NaN   
6    Winter Storm         Alabama  darrellmansfield             NaN   
7    Winter Storm         Alabama  darrellmansfield             NaN   
8      Waterspout       E pacific        rentclicks             NaN   
9      Waterspout       E pacific        rentclicks             NaN   

                        SEARCH_QUERY  CATEGORY_TOPIC  
0          email bosanska kostajnica             NaN  
1          email bosanska kostajnica             NaN  
2          emai

In [27]:
query_facts_keywords = """
SELECT
    w.EVENT_TYPE,
    w.REGION,
    u.THISDOMAIN AS DOMAIN_NAME,
    u.TITLE AS WEBSITE_TITLE,
    q.QUERY AS SEARCH_QUERY,
    c.TOPIC AS CATEGORY_TOPIC
FROM
    AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN
    AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
LEFT JOIN
    AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
LEFT JOIN
    AOL_SCHEMA.QUERYDIM q ON f.QUERYID = q.ID
LEFT JOIN
    AOL_SCHEMA.DMOZ_CATEGORIES c ON u.THISDOMAIN = c.TOPIC;
"""
df_facts_keywords = conn.export_to_pandas(query_facts_keywords)
print("Website Analysis with Event Keywords:")
print(df_facts_keywords)

Website Analysis with Event Keywords:
           EVENT_TYPE          REGION       DOMAIN_NAME   WEBSITE_TITLE  \
0      Winter Weather  North carolina        kostajnica  Kostajnica.com   
1      Winter Weather  North carolina        kostajnica  Kostajnica.com   
2      Winter Weather  North carolina        kostajnica  Kostajnica.com   
3      Winter Weather  North carolina        kostajnica  Kostajnica.com   
4        Winter Storm         Alabama  darrellmansfield             NaN   
...               ...             ...               ...             ...   
82628      Heavy Snow      Washington       safeshopper             NaN   
82629      Heavy Snow      Washington       safeshopper             NaN   
82630      Heavy Snow      Washington       safeshopper             NaN   
82631      Heavy Snow      Washington       safeshopper             NaN   
82632      Heavy Snow      Washington       safeshopper             NaN   

                    SEARCH_QUERY  CATEGORY_TOPIC  
0      ema

In [32]:
# Remove duplicates
df_cleaned = df_facts_keywords.drop_duplicates()
print("Data after removing duplicates:")
print(df_cleaned)

Data after removing duplicates:
           EVENT_TYPE          REGION       DOMAIN_NAME   WEBSITE_TITLE  \
0      Winter Weather  North carolina        kostajnica  Kostajnica.com   
4        Winter Storm         Alabama  darrellmansfield             NaN   
8          Waterspout       E pacific        rentclicks             NaN   
9          Waterspout       E pacific        rentclicks             NaN   
10     Winter Weather        Virginia             sunoa             NaN   
...               ...             ...               ...             ...   
82576     Strong Wind          Kansas             crime             NaN   
82578     Strong Wind          Kansas             crime             NaN   
82580     Strong Wind          Kansas             crime             NaN   
82581     Strong Wind          Kansas             crime             NaN   
82582     Strong Wind          Kansas             crime             NaN   

                            SEARCH_QUERY  CATEGORY_TOPIC  
0       

In [33]:
from collections import Counter

# Combine SEARCH_QUERY and DOMAIN_NAME columns into one
df_cleaned['ALL_TEXT'] = df_cleaned['SEARCH_QUERY'].fillna('') + ' ' + df_cleaned['DOMAIN_NAME'].fillna('')

# Tokenize words (split by whitespace)
all_words = ' '.join(df_cleaned['ALL_TEXT']).lower().split()


C:\Users\Anwender\AppData\Local\Temp\ipykernel_20808\2860728013.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['ALL_TEXT'] = df_cleaned['SEARCH_QUERY'].fillna('') + ' ' + df_cleaned['DOMAIN_NAME'].fillna('')


In [37]:
# Count word frequencies
word_counts = Counter(all_words)

# Convert to a DataFrame for better visualization
word_count_df = pd.DataFrame(word_counts.items(), columns=['Word', 'Frequency'])

# Sort by frequency
word_count_df = word_count_df.sort_values(by='Frequency', ascending=False)

# print(word_count_df.head(20))  # Show the top 20 words
# Define a list of stop words
stop_words = {'the', 'and', 'of', 'to', 'a', 'in', 'for', 'is', 'on', 'with', 'at', 'by', 'from', 'an', '-', '3'}

# Filter out stop words
word_count_df = word_count_df[~word_count_df['Word'].isin(stop_words)]

print(word_count_df.head(20))  # Show the top 20 meaningful words


                  Word  Frequency
171        lvbeethoven        378
95          watsonrent        272
411              crime        234
284      childalert.co        210
66      kingdomhearts3        160
168          beethoven        144
141           mybrotha        120
227  bentleypublishers        108
258       bookooenergy         84
142              black         82
431     buildingtrades         81
67             kingdom         80
96              watson         80
394   elliscountypress         80
143                men         80
100            florida         64
92            property         64
114             realty         64
64              hearts         64
38            magelang         60


In [47]:
query_region = """
SELECT 
    w.REGION,
    COUNT(f.QUERYID) AS total_queries
FROM AOL_SCHEMA.WEATHER_EVENTS w
JOIN AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID  -- Ensure STATE_FIPS and URLID match logically
GROUP BY w.REGION
ORDER BY total_queries DESC
LIMIT 10;
"""
df_region = conn.export_to_pandas(query_region)
print("Query Volume by Region:")
print(df_region)


Query Volume by Region:
      REGION  TOTAL_QUERIES
0     Kansas          11946
1  Tennessee          10092
2   Michigan           7803
3   Illinois           7266
4     Oregon           5751
5    Indiana           4150
6  Wisconsin           3705
7   Kentucky           3308
8    Alabama           2604
9   Arkansas           1944


In [51]:
query = """
SELECT DISTINCT u.THISDOMAIN, c.TOPIC
FROM AOL_SCHEMA.URLDIM u
LEFT JOIN AOL_SCHEMA.DMOZ_CATEGORIES c
ON u.THISDOMAIN = c.TOPIC
WHERE c.TOPIC IS NOT NULL
LIMIT 10;
"""
print(conn.export_to_pandas(query))


Empty DataFrame
Columns: [THISDOMAIN, TOPIC]
Index: []


In [50]:
# Define the SQL query
query = """
SELECT
    w.EVENT_TYPE,
    w.REGION,
    CASE 
        WHEN c.TITLE IS NULL THEN 'Uncategorized'
        ELSE c.TITLE
    END AS WEBSITE_CATEGORY,
    SUM(w.DAMAGE_PROPERTY) AS TOTAL_PROPERTY_DAMAGE,
    SUM(w.DEATHS_DIRECT + w.DEATHS_INDIRECT) AS TOTAL_FATALITIES,
    COUNT(f.QUERYID) AS TOTAL_QUERIES,
    GROUPING(w.EVENT_TYPE) AS GROUPING_EVENT_TYPE,
    GROUPING(w.REGION) AS GROUPING_REGION,
    GROUPING(c.TITLE) AS GROUPING_WEBSITE_CATEGORY
FROM
    AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN
    AOL_SCHEMA.FACTS f
ON
    w.STATE_FIPS = f.URLID -- Assuming URLID maps to STATE_FIPS
LEFT JOIN
    AOL_SCHEMA.URLDIM u
ON
    f.URLID = u.ID
LEFT JOIN
    AOL_SCHEMA.DMOZ_CATEGORIES c
ON
    u.THISDOMAIN = c.TOPIC
GROUP BY GROUPING SETS (
    (w.EVENT_TYPE, w.REGION, c.TITLE),
    (w.EVENT_TYPE, w.REGION),
    (w.EVENT_TYPE),
    ()
)
HAVING
    SUM(w.DAMAGE_PROPERTY) > 100000 OR
    SUM(w.DEATHS_DIRECT + w.DEATHS_INDIRECT) > 5
ORDER BY
    TOTAL_PROPERTY_DAMAGE DESC,
    TOTAL_FATALITIES DESC,
    TOTAL_QUERIES DESC;
"""
print(conn.export_to_pandas(query))
# result = conn.execute(query)

# # Fetch and print the results
# for row in result:
#     print(row)

            EVENT_TYPE   REGION WEBSITE_CATEGORY  TOTAL_PROPERTY_DAMAGE  \
0                  NaN      NaN    Uncategorized               82633000   
1                 Hail      NaN    Uncategorized               37350000   
2    Thunderstorm Wind      NaN    Uncategorized               18939000   
3                 Hail   Kansas    Uncategorized                8195000   
4                 Hail   Kansas    Uncategorized                8195000   
..                 ...      ...              ...                    ...   
266          Avalanche  Montana    Uncategorized                   4000   
267        Strong Wind  Arizona    Uncategorized                   4000   
268          Avalanche  Montana    Uncategorized                   4000   
269       Winter Storm  Arizona    Uncategorized                   4000   
270       Winter Storm  Arizona    Uncategorized                   4000   

     TOTAL_FATALITIES  TOTAL_QUERIES  GROUPING_EVENT_TYPE  GROUPING_REGION  \
0                 837

In [6]:
# Define the SQL query
query = """
SELECT
    w.EVENT_TYPE,
    w.REGION,
    u.THISDOMAIN AS DOMAIN_NAME,
    u.TITLE AS WEBSITE_TITLE,
    SUM(w.DAMAGE_PROPERTY) AS TOTAL_PROPERTY_DAMAGE,
    SUM(w.DEATHS_DIRECT + w.DEATHS_INDIRECT) AS TOTAL_FATALITIES,
    COUNT(f.QUERYID) AS TOTAL_QUERIES,
    GROUPING(w.EVENT_TYPE) AS GROUPING_EVENT_TYPE,
    GROUPING(w.REGION) AS GROUPING_REGION,
    GROUPING(u.THISDOMAIN) AS GROUPING_DOMAIN_NAME
FROM
    AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN
    AOL_SCHEMA.FACTS f
ON
    w.STATE_FIPS = f.URLID -- Assuming URLID maps to STATE_FIPS
LEFT JOIN
    AOL_SCHEMA.URLDIM u
ON
    f.URLID = u.ID
GROUP BY GROUPING SETS (
    (w.EVENT_TYPE, w.REGION, u.THISDOMAIN, u.TITLE),
    (w.EVENT_TYPE, w.REGION, u.THISDOMAIN),
    (w.EVENT_TYPE, w.REGION),
    (w.EVENT_TYPE),
    ()
)
HAVING
    SUM(w.DAMAGE_PROPERTY) > 100000 OR
    SUM(w.DEATHS_DIRECT + w.DEATHS_INDIRECT) > 5
ORDER BY
    TOTAL_PROPERTY_DAMAGE DESC,
    TOTAL_FATALITIES DESC,
    TOTAL_QUERIES DESC
LIMIT 20;
"""
# Assume df is the DataFrame containing the query results
df = conn.export_to_pandas(query)
filtered_df = df.dropna()
filtered_df

,EVENT_TYPE,REGION,DOMAIN_NAME,WEBSITE_TITLE,TOTAL_PROPERTY_DAMAGE,TOTAL_FATALITIES,TOTAL_QUERIES,GROUPING_EVENT_TYPE,GROUPING_REGION,GROUPING_DOMAIN_NAME
13,Hail,Illinois,mybrotha,Mybrotha.com,3666000,0,3666,0,0,0


In [52]:
# Define the SQL query
query = """
SELECT 
    w.REGION,
    w.EVENT_TYPE,
    u.TITLE AS WEBSITE_TITLE,
    COUNT(f.QUERYID) AS TOTAL_QUERIES,
    GROUPING(w.REGION) AS GROUPING_REGION,
    GROUPING(w.EVENT_TYPE) AS GROUPING_EVENT_TYPE,
    GROUPING(u.TITLE) AS GROUPING_WEBSITE_TITLE
FROM AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
LEFT JOIN AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
GROUP BY GROUPING SETS (
    (w.REGION, w.EVENT_TYPE, u.TITLE),
    (w.REGION, w.EVENT_TYPE),
    (w.EVENT_TYPE),
    ()
)
ORDER BY TOTAL_QUERIES DESC
LIMIT 20;

"""
print(conn.export_to_pandas(query))
# result = conn.execute(query)

# # Fetch and print the results
# for row in result:
#     print(row)

       REGION         EVENT_TYPE       WEBSITE_TITLE  TOTAL_QUERIES  \
0         NaN                NaN                 NaN          82633   
1         NaN               Hail                 NaN          37350   
2         NaN  Thunderstorm Wind                 NaN          18939   
3      Kansas               Hail                 NaN           8195   
4      Kansas               Hail                 NaN           8195   
5         NaN          High Wind                 NaN           5004   
6   Tennessee               Hail                 NaN           4908   
7   Tennessee               Hail                 NaN           4908   
8   Tennessee  Thunderstorm Wind                 NaN           4056   
9   Tennessee  Thunderstorm Wind                 NaN           4056   
10   Illinois               Hail        Mybrotha.com           3666   
11   Illinois               Hail                 NaN           3666   
12        NaN         Heavy Snow                 NaN           3593   
13    

In [31]:
# Define the SQL query
query = """
SELECT 
    w.STATE_FIPS,
    w.REGION,
    c.TITLE AS WEBSITE_CATEGORY,
    SUM(w.DAMAGE_PROPERTY) AS TOTAL_PROPERTY_DAMAGE,
    COUNT(f.QUERYID) AS TOTAL_QUERIES,
    GROUPING(w.STATE_FIPS) AS GROUPING_STATE_FIPS,
    GROUPING(c.TITLE) AS GROUPING_WEBSITE_CATEGORY
FROM AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
LEFT JOIN AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
LEFT JOIN AOL_SCHEMA.DMOZ_CATEGORIES c ON u.THISDOMAIN = REGEXP_SUBSTR(c.TOPIC, '[^/]+$', 1, 1)
GROUP BY GROUPING SETS (
    (w.STATE_FIPS, w.REGION, c.TITLE),
    (w.STATE_FIPS, c.TITLE),
    (w.STATE_FIPS),
    ()
)
HAVING SUM(w.DAMAGE_PROPERTY) > 1000000
ORDER BY TOTAL_PROPERTY_DAMAGE DESC
LIMIT 20;
"""
print(conn.export_to_pandas(query))
# result = conn.execute(query)

# # Fetch and print the results
# for row in result:
#     print(row)

    STATE_FIPS     REGION  WEBSITE_CATEGORY  TOTAL_PROPERTY_DAMAGE  \
0          NaN        NaN               NaN               82633000   
1         20.0     Kansas               NaN               11946000   
2         20.0        NaN               NaN               11946000   
3         20.0        NaN               NaN               11946000   
4         47.0  Tennessee               NaN               10092000   
5         47.0        NaN               NaN               10092000   
6         47.0        NaN               NaN               10092000   
7         26.0   Michigan               NaN                7803000   
8         26.0        NaN               NaN                7803000   
9         26.0        NaN               NaN                7803000   
10        17.0   Illinois               NaN                7266000   
11        17.0        NaN               NaN                7266000   
12        17.0        NaN               NaN                7266000   
13        41.0     O

In [54]:
# Define the SQL query
query = """
SELECT 
    w.EVENT_TYPE,
    w.REGION,
    f.ANONID AS DEMOGRAPHIC_GROUP,
    SUM(w.DEATHS_DIRECT + w.DEATHS_INDIRECT) AS TOTAL_FATALITIES,
    COUNT(f.QUERYID) AS TOTAL_QUERIES,
    RANK() OVER (PARTITION BY w.EVENT_TYPE ORDER BY SUM(w.DEATHS_DIRECT + w.DEATHS_INDIRECT) DESC) AS FATALITY_RANK
FROM AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
GROUP BY w.EVENT_TYPE, w.REGION, f.ANONID
HAVING SUM(w.DEATHS_DIRECT + w.DEATHS_INDIRECT) > 5
ORDER BY FATALITY_RANK, TOTAL_FATALITIES DESC, TOTAL_QUERIES DESC
LIMIT 20;
"""
print(conn.export_to_pandas(query))
# result = conn.execute(query)

# # Fetch and print the results
# for row in result:
#     print(row)

     EVENT_TYPE      REGION  DEMOGRAPHIC_GROUP  TOTAL_FATALITIES  \
0       Tornado   Tennessee             444221               102   
1      Wildfire       Texas             466967                12   
2     High Surf      Oregon             518451                12   
3   Flash Flood      Hawaii              84569                 7   
4   Strong Wind  Washington             528758                 6   
5     High Wind    Michigan             139958                 6   
6     Avalanche  Washington             528758                 6   
7       Tornado   Tennessee             519991                68   
8     High Surf      Oregon             203495                 9   
9       Tornado   Tennessee             189208                34   
10      Tornado   Tennessee             486002                34   
11      Tornado   Tennessee             359602                34   
12      Tornado   Tennessee             649329                34   
13      Tornado   Tennessee             180831  

In [21]:
# Define the SQL query
query = """
SELECT 
    w.REGION,
    u.THISDOMAIN AS DOMAIN_NAME,
    SUM(w.INJURIES_DIRECT + w.INJURIES_INDIRECT) AS TOTAL_INJURIES,
    COUNT(f.QUERYID) AS TOTAL_QUERIES,
    RANK() OVER (PARTITION BY w.REGION ORDER BY SUM(w.INJURIES_DIRECT + w.INJURIES_INDIRECT) DESC) AS INJURY_RANK
FROM AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
LEFT JOIN AOL_SCHEMA.URLDIM u ON f.URLID = u.ID
GROUP BY w.REGION, u.THISDOMAIN
HAVING SUM(w.INJURIES_DIRECT + w.INJURIES_INDIRECT) > 10
ORDER BY INJURY_RANK, TOTAL_QUERIES DESC
LIMIT 10;
"""
print(conn.export_to_pandas(query))
# result = conn.execute(query)

# # Fetch and print the results
# for row in result:
#     print(row)

      REGION       DOMAIN_NAME  TOTAL_INJURIES  TOTAL_QUERIES  INJURY_RANK
0     Kansas             crime             209          11946            1
1  Tennessee    buildingtrades            3168          10092            1
2   Michigan       lvbeethoven            1566           7803            1
3   Illinois          mybrotha             384           7266            1
4     Oregon        watsonrent              54           5751            1
5    Indiana  elliscountypress              30           4150            1
6  Wisconsin    kingdomhearts3             105           3705            1
7   Kentucky          magelang             156           3308            1
8    Alabama  darrellmansfield              48           2604            1
9   Arkansas             mxram             219           1944            1


In [20]:
# Define the SQL query
query = """
SELECT 
    TO_CHAR(w.BEGIN_DATE_TIME, 'YYYY-MM') AS YEAR_MONTH,
    w.EVENT_TYPE,
    COUNT(f.QUERYID) AS TOTAL_QUERIES,
    SUM(w.DAMAGE_PROPERTY) AS TOTAL_PROPERTY_DAMAGE,
    GROUPING(TO_CHAR(w.BEGIN_DATE_TIME, 'YYYY-MM')) AS GROUPING_MONTH,
    GROUPING(w.EVENT_TYPE) AS GROUPING_EVENT_TYPE
FROM AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
GROUP BY ROLLUP (
    TO_CHAR(w.BEGIN_DATE_TIME, 'YYYY-MM'),
    w.EVENT_TYPE
)
ORDER BY YEAR_MONTH ASC, TOTAL_QUERIES DESC;
"""
print(conn.export_to_pandas(query))
# result = conn.execute(query)

# # Fetch and print the results
# for row in result:
#     print(row)

    YEAR_MONTH        EVENT_TYPE  TOTAL_QUERIES  TOTAL_PROPERTY_DAMAGE  \
0      2006-01               NaN           7059                7059000   
1      2006-01         High Wind           2209                2209000   
2      2006-01        Heavy Snow           1130                1130000   
3      2006-01              Hail            567                 567000   
4      2006-01      Winter Storm            510                 510000   
..         ...               ...            ...                    ...   
146    2006-05     Coastal Flood              5                   5000   
147    2006-05   Cold/Wind Chill              2                   2000   
148    2006-05  Marine High Wind              2                   2000   
149    2006-05         Avalanche              1                   1000   
150        NaN               NaN          82633               82633000   

     GROUPING_MONTH  GROUPING_EVENT_TYPE  
0                 0                    1  
1                 0      

In [56]:
# Define the SQL query
query = """
SELECT 
    w.EVENT_TYPE,
    f.QUERYID,
    COUNT(f.QUERYID) AS TOTAL_QUERIES,
    SUM(w.DAMAGE_CROPS) AS TOTAL_CROP_DAMAGE,
    RANK() OVER (PARTITION BY w.EVENT_TYPE ORDER BY SUM(w.DAMAGE_CROPS) DESC) AS CROP_DAMAGE_RANK
FROM AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
GROUP BY w.EVENT_TYPE, f.QUERYID
HAVING SUM(w.DAMAGE_CROPS) > 50000
ORDER BY CROP_DAMAGE_RANK, TOTAL_CROP_DAMAGE DESC, TOTAL_QUERIES DESC
LIMIT 5;
"""
print(conn.export_to_pandas(query))
# result = conn.execute(query)

# # Fetch and print the results
# for row in result:
#     print(row)

    EVENT_TYPE  QUERYID  TOTAL_QUERIES  TOTAL_CROP_DAMAGE  CROP_DAMAGE_RANK
0      Drought  4522325             26         1600250000                 1
1     Wildfire  4522325             87          139070000                 1
2   Heavy Rain   371707             82           93020000                 1
3        Flood   371707             66           84600000                 1
4  Debris Flow   371707             24           40000000                 1


In [23]:
# Define the SQL query
query = """
SELECT 
    w.EVENT_TYPE,
    q.QUERY AS SEARCH_QUERY,
    COUNT(f.QUERYID) AS TOTAL_QUERIES,
    SUM(w.DAMAGE_CROPS) AS TOTAL_CROP_DAMAGE,
    RANK() OVER (PARTITION BY w.EVENT_TYPE ORDER BY SUM(w.DAMAGE_CROPS) DESC) AS CROP_DAMAGE_RANK
FROM AOL_SCHEMA.WEATHER_EVENTS w
LEFT JOIN AOL_SCHEMA.FACTS f ON w.STATE_FIPS = f.URLID
LEFT JOIN AOL_SCHEMA.QUERYDIM q ON f.QUERYID = q.ID  -- Replace QUERYID with the correct column in QUERYDIM
GROUP BY w.EVENT_TYPE, q.QUERY
HAVING SUM(w.DAMAGE_CROPS) > 50000
ORDER BY CROP_DAMAGE_RANK, TOTAL_CROP_DAMAGE DESC, TOTAL_QUERIES DESC
LIMIT 10;
"""
print(conn.export_to_pandas(query))
# result = conn.execute(query)

# Fetch and print the results
# for row in result:
#     print(row)

          EVENT_TYPE                 SEARCH_QUERY  TOTAL_QUERIES  \
0            Drought                fortunate son             26   
1           Wildfire                fortunate son             87   
2         Heavy Rain  www.larrywilliamsphotog.com             82   
3              Flood  www.larrywilliamsphotog.com             66   
4        Debris Flow  www.larrywilliamsphotog.com             24   
5       Frost/Freeze  www.larrywilliamsphotog.com             32   
6               Hail                 www.free4all            546   
7            Tornado                       nastya             13   
8  Thunderstorm Wind          beethoven biography            414   
9        Strong Wind  www.larrywilliamsphotog.com             94   

   TOTAL_CROP_DAMAGE  CROP_DAMAGE_RANK  
0         1600250000                 1  
1          139070000                 1  
2           93020000                 1  
3           84600000                 1  
4           40000000                 1  
5    

In [59]:
query = """
DESCRIBE AOL_SCHEMA.QUERYDIM;
"""
# print(conn.export_to_pandas(query))
result = conn.execute(query)
for row in result:
    print(row)

('QUERY', 'VARCHAR(5000) UTF8', 'TRUE', 'FALSE')
('ID', 'DECIMAL(18,0)', 'TRUE', 'FALSE')


In [33]:
# Execute the query and fetch results into a Pandas DataFrame
df = conn.export_to_pandas(query)
df

,EVENT_TYPE,REGION,WEBSITE_CATEGORY,TOTAL_PROPERTY_DAMAGE,TOTAL_FATALITIES,TOTAL_QUERIES


In [38]:
conn.close()